In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

##Download the data and read it into a pandas dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


###The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [3]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

##Normalize Data

In [4]:
predictors_norm = (predictors - predictors.mean()) / predictors.std() #normalize the data by substracting the mean and dividing by the standard deviation.
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [5]:
n_cols = predictors_norm.shape[1] # save number of predictors to n_cols since we will need this number when building our network
n_cols

8

## Use the Keras library to build a neural network

In [6]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,))) # Add the hidden layer with 10 nodes and ReLU activation function
    model.add(Dense(1)) # Output layer with one node

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [7]:
model = regression_model() #build the model

##1. Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [8]:
X = predictors_norm  # All columns except Strength
y = target  # Strength column

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

##2. Train the model on the training data using 50 epochs.

In [10]:
# Train the model on the training data
model.fit(X_train, y_train, epochs=50, verbose=2)  # Training with 50 epochs

Epoch 1/50
23/23 - 2s - loss: 1602.2507 - 2s/epoch - 90ms/step
Epoch 2/50
23/23 - 0s - loss: 1585.5577 - 67ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 1568.9601 - 59ms/epoch - 3ms/step
Epoch 4/50
23/23 - 0s - loss: 1552.2520 - 77ms/epoch - 3ms/step
Epoch 5/50
23/23 - 0s - loss: 1535.2739 - 85ms/epoch - 4ms/step
Epoch 6/50
23/23 - 0s - loss: 1517.9213 - 55ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1499.8318 - 55ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 1481.0743 - 51ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 1461.6737 - 52ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1441.1990 - 98ms/epoch - 4ms/step
Epoch 11/50
23/23 - 0s - loss: 1420.0597 - 71ms/epoch - 3ms/step
Epoch 12/50
23/23 - 0s - loss: 1397.8113 - 49ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 1374.7349 - 58ms/epoch - 3ms/step
Epoch 14/50
23/23 - 0s - loss: 1350.7067 - 64ms/epoch - 3ms/step
Epoch 15/50
23/23 - 0s - loss: 1325.9618 - 53ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 1299

##3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [11]:
# Predict on test data
y_pred = model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

10/10 [==============================] - 0s 2ms/step


##4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [12]:
num_repeats = 50
mse_list = []

for _ in range(num_repeats):
    # Step 1: Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # we call the contructor to initialize the weights (fit method DOES NOT initialize weights)
    model = regression_model() #build the model

    # Step 2: Train the model
    model.fit(X_train, y_train, epochs=50, verbose=2)  # Training with 50 epochs

    # Step 3: Evaluate the model and compute mean squared error
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

Streaming output truncated to the last 5000 lines.
23/23 - 0s - loss: 879.5132 - 75ms/epoch - 3ms/step
Epoch 27/50
23/23 - 0s - loss: 847.0996 - 91ms/epoch - 4ms/step
Epoch 28/50
23/23 - 0s - loss: 815.1967 - 75ms/epoch - 3ms/step
Epoch 29/50
23/23 - 0s - loss: 783.9478 - 79ms/epoch - 3ms/step
Epoch 30/50
23/23 - 0s - loss: 753.8443 - 53ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 723.6624 - 113ms/epoch - 5ms/step
Epoch 32/50
23/23 - 0s - loss: 694.4329 - 60ms/epoch - 3ms/step
Epoch 33/50
23/23 - 0s - loss: 666.2634 - 51ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 638.9557 - 52ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 612.3272 - 107ms/epoch - 5ms/step
Epoch 36/50
23/23 - 0s - loss: 586.7919 - 84ms/epoch - 4ms/step
Epoch 37/50
23/23 - 0s - loss: 562.4918 - 90ms/epoch - 4ms/step
Epoch 38/50
23/23 - 0s - loss: 539.0063 - 64ms/epoch - 3ms/step
Epoch 39/50
23/23 - 0s - loss: 516.7283 - 77ms/epoch - 3ms/step
Epoch 40/50
23/23 - 0s - loss: 495.5300 - 67ms/epoch - 3ms/step

##5. Report the mean and the standard deviation of the mean squared errors.

In [13]:
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print("Mean of Mean Squared Errors over 50 repeats:", mean_mse)
print("Standard Deviation of Mean Squared Errors over 50 repeats:", std_mse)

Mean of Mean Squared Errors over 50 repeats: 333.7483045623653
Standard Deviation of Mean Squared Errors over 50 repeats: 80.70528877132512


#C : Part B but with 100 epochs for training

##Build a new network

In [14]:
model1 = regression_model() #build the model

##1. Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [15]:
X = predictors_norm  # All columns except Strength
y = target  # Strength column

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

##2. Train the model on the training data using 100 epochs.

In [17]:
# Train the model on the training data
model1.fit(X_train, y_train, epochs=100, verbose=2)  # Training with 100 epochs

Epoch 1/100
23/23 - 1s - loss: 1541.3086 - 512ms/epoch - 22ms/step
Epoch 2/100
23/23 - 0s - loss: 1523.1208 - 32ms/epoch - 1ms/step
Epoch 3/100
23/23 - 0s - loss: 1504.4885 - 31ms/epoch - 1ms/step
Epoch 4/100
23/23 - 0s - loss: 1485.7578 - 36ms/epoch - 2ms/step
Epoch 5/100
23/23 - 0s - loss: 1466.2098 - 31ms/epoch - 1ms/step
Epoch 6/100
23/23 - 0s - loss: 1445.8251 - 40ms/epoch - 2ms/step
Epoch 7/100
23/23 - 0s - loss: 1424.9067 - 32ms/epoch - 1ms/step
Epoch 8/100
23/23 - 0s - loss: 1402.5784 - 34ms/epoch - 1ms/step
Epoch 9/100
23/23 - 0s - loss: 1379.5684 - 32ms/epoch - 1ms/step
Epoch 10/100
23/23 - 0s - loss: 1355.9069 - 32ms/epoch - 1ms/step
Epoch 11/100
23/23 - 0s - loss: 1330.7815 - 34ms/epoch - 1ms/step
Epoch 12/100
23/23 - 0s - loss: 1305.1144 - 33ms/epoch - 1ms/step
Epoch 13/100
23/23 - 0s - loss: 1277.8740 - 35ms/epoch - 2ms/step
Epoch 14/100
23/23 - 0s - loss: 1250.3378 - 37ms/epoch - 2ms/step
Epoch 15/100
23/23 - 0s - loss: 1221.6351 - 32ms/epoch - 1ms/step
Epoch 16/100
23/2

##3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [18]:
# Predict on test data
y_pred = model1.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

10/10 [==============================] - 0s 2ms/step


##4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [19]:
num_repeats = 50
mse_list1 = []

for _ in range(num_repeats):
    # Step 1: Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model1 = regression_model() #build the model
    # Step 2: Train the model
    model1.fit(X_train, y_train, epochs=100, verbose=2)  # Training with 100 epochs

    # Step 3: Evaluate the model and compute mean squared error
    y_pred = model1.predict(X_test)
    mse1 = mean_squared_error(y_test, y_pred)
    mse_list1.append(mse)

Streaming output truncated to the last 5000 lines.
Epoch 14/100
23/23 - 0s - loss: 1479.1115 - 45ms/epoch - 2ms/step
Epoch 15/100
23/23 - 0s - loss: 1464.4733 - 41ms/epoch - 2ms/step
Epoch 16/100
23/23 - 0s - loss: 1448.0840 - 38ms/epoch - 2ms/step
Epoch 17/100
23/23 - 0s - loss: 1429.9196 - 39ms/epoch - 2ms/step
Epoch 18/100
23/23 - 0s - loss: 1409.8223 - 38ms/epoch - 2ms/step
Epoch 19/100
23/23 - 0s - loss: 1387.7717 - 41ms/epoch - 2ms/step
Epoch 20/100
23/23 - 0s - loss: 1363.9076 - 37ms/epoch - 2ms/step
Epoch 21/100
23/23 - 0s - loss: 1338.3667 - 38ms/epoch - 2ms/step
Epoch 22/100
23/23 - 0s - loss: 1311.1652 - 41ms/epoch - 2ms/step
Epoch 23/100
23/23 - 0s - loss: 1282.5538 - 55ms/epoch - 2ms/step
Epoch 24/100
23/23 - 0s - loss: 1252.3752 - 40ms/epoch - 2ms/step
Epoch 25/100
23/23 - 0s - loss: 1220.6621 - 37ms/epoch - 2ms/step
Epoch 26/100
23/23 - 0s - loss: 1187.3964 - 41ms/epoch - 2ms/step
Epoch 27/100
23/23 - 0s - loss: 1152.6195 - 38ms/epoch - 2ms/step
Epoch 28/100
23/23 - 0s -

##5. Report the mean and the standard deviation of the mean squared errors.

In [20]:
mean_mse = np.mean(mse_list1)
std_mse = np.std(mse_list1)

print("Mean of Mean Squared Errors over 100 repeats:", mean_mse)
print("Standard Deviation of Mean Squared Errors over 100 repeats:", std_mse)

Mean of Mean Squared Errors over 100 repeats: 151.44160974396686
Standard Deviation of Mean Squared Errors over 100 repeats: 2.842170943040401e-14


#D : Part B but increase Number of Layers

##Build a Neural Network

In [21]:
def regression_model():
    model2 = Sequential()
    model2.add(Dense(10, activation='relu', input_shape=(n_cols,)))  # 1st hidden layer with 10 nodes
    model2.add(Dense(10, activation='relu'))  # 2nd hidden layer with 10 nodes
    model2.add(Dense(10, activation='relu'))  # 3rd hidden layer with 10 nodes
    model2.add(Dense(1))  # Output layer
    model2.compile(optimizer='adam', loss='mean_squared_error')
    return model2

In [22]:
model2 = regression_model() #build the model

##1. Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [23]:
X = predictors_norm  # All columns except Strength
y = target  # Strength column

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

##2. Train the model on the training data using 50 epochs.

In [25]:
# Train the model on the training data
model2.fit(X_train, y_train, epochs=50, verbose=2)  # Training with 50 epochs

Epoch 1/50
23/23 - 1s - loss: 1546.7314 - 1s/epoch - 47ms/step
Epoch 2/50
23/23 - 0s - loss: 1510.0714 - 52ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 1457.6411 - 50ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 1379.6848 - 55ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 1268.5729 - 45ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1118.1691 - 43ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 922.7728 - 55ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 706.3585 - 53ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 508.6655 - 52ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 375.0808 - 49ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 317.2209 - 46ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 286.4798 - 48ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 268.8155 - 46ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 253.9879 - 53ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 242.0391 - 69ms/epoch - 3ms/step
Epoch 16/50
23/23 - 0s - loss: 232.3315 - 56

##3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [26]:
# Predict on test data
y_pred = model2.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)

10/10 [==============================] - 0s 2ms/step


##4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [27]:
num_repeats = 50
mse_list2 = []

for _ in range(num_repeats):
    # Step 1: Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model2 = regression_model() #build the model
    # Step 2: Train the model
    model2.fit(X_train, y_train, epochs=50, verbose=2)  # Training with 100 epochs

    # Step 3: Evaluate the model and compute mean squared error
    y_pred = model2.predict(X_test)
    mse2 = mean_squared_error(y_test, y_pred)
    mse_list2.append(mse)

Streaming output truncated to the last 5000 lines.
23/23 - 0s - loss: 165.5454 - 41ms/epoch - 2ms/step
Epoch 27/50
23/23 - 0s - loss: 163.3064 - 37ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 160.7576 - 41ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 158.3978 - 48ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 156.7431 - 37ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 154.4387 - 39ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 152.9106 - 42ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 150.8529 - 37ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 149.6050 - 39ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 147.7374 - 41ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 146.6260 - 47ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 144.6810 - 42ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 144.0838 - 39ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 142.6095 - 43ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 140.7204 - 39ms/epoch - 2ms/step
E

##5. Report the mean and the standard deviation of the mean squared errors.

In [28]:
mean_mse = np.mean(mse_list2)
std_mse = np.std(mse_list2)

print("Mean of Mean Squared Errors over 50 repeats:", mean_mse)
print("Standard Deviation of Mean Squared Errors over 50 repeats:", std_mse)

Mean of Mean Squared Errors over 50 repeats: 139.15198117692304
Standard Deviation of Mean Squared Errors over 50 repeats: 2.842170943040401e-14
